# Phenotype data formatting


This module implements a collection of workflows used to format molecular phenotype data.



## Input
The input for this workflow is the collection of data for 1 conditions as described in the readme of this git repo
1. 1 complete residual molecular_phenotype data
2. 1 region_list
Both of these input can be generated by the annotation module of this pipeline

## Output
For each collection, the output is 
1. 1 lists of phenotype file (bed+index) for each chrom, suitable to be fed into both apex and tensorQTL, annotated with chrom and pos
2. 1 lists of phenotype file (bed+index) for each gene, annotated with chrom and tss

## Minimal working example
An MWE is uploaded to [google drive](https://drive.google.com/drive/folders/1yjTwoO0DYGi-J9ouMsh9fHKfDmsXJ_4I?usp=sharing).
The singularity image (sif) for running this MWE is uploaded to [google drive](https://drive.google.com/drive/folders/1mLOS3AVQM8yTaWtCbO8Q3xla98Nr5bZQ)


In [ ]:
sos run pipeline/phenotype_formatting.ipynb partition_by_chrom \
    --cwd output  \
    --phenoFile MWE.log2cpm.mol_phe.bed.gz \
    --region-list MWE.region_list \
    --container containers/rna_quantification.sif

In [2]:
[global]
import os
# Work directory & output directory
parameter: cwd = path("output")
# The filename namefor output data
parameter: container = ''

# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
# Path to the input molecular phenotype data.
parameter: phenoFile = paths
# name for the analysis output
parameter: name= f'{phenoFile:bn}'
# Whether the input data is named by gene_id or gene_name. By default it is gene_id, if not, please change it to gene_name
parameter: phenotype_id_type = 'gene_id'
gene_name_as_phenotype_id = "gene_name" == phenotype_id_type
parameter: region_list = path
import pandas as pd
regions = [x.strip().split() for x in open(region_list).readlines() if x.strip() and not x.strip().startswith('#')]
# Get the unique chormosome that have regions to be analyzed.
def extract(lst):
    return [item[0] for item in lst]
chrom = list(set(extract(regions)))

## Region List generation

To partitioning the data by genes require a region list file which:

    1. have 5 columns: chr,start,end,gene_id,gene_name
    2. have the same gene as or less gene than that of the bed file
    
Input:

    1. A gtf file used to generated the bed
    2. A phenotype bed file, must have a gene_id column indicating the name of genes.    

In [ ]:
[generate_region_list]
#  gene gtf annotation table
parameter: annotation_gtf = path
input: phenoFile, annotation_gtf
output: f'{cwd}/{_input[0]:bnn}.region_list'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime,  mem = mem, tags = f'{step_name}_{_output:bn}'  
python: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container
    import pandas as pd
    import qtl.io
    # get the five column data
    bed_template_df_id = qtl.io.gtf_to_tss_bed(${_input[1]:r}, feature='transcript',phenotype_id = "gene_id" )
    bed_template_df_name = qtl.io.gtf_to_tss_bed(${_input[1]:r}, feature='transcript',phenotype_id = "gene_name" )
    bed_template_df = bed_template_df_id.merge(bed_template_df_name, on = ["chr","start","end"])
    bed_template_df.columns = ["#chr","start","end","gene_id","gene_name"]
    pheno = pd.read_csv(${_input[0]:r}, sep = "\t")
    # Retaining only the genes in the data
    region_list = bed_template_df[bed_template_df.${phenotype_id_type}.isin(pheno.gene_id)]
    region_list.to_csv("${_output}", sep = "\t",index = 0)

## Process of molecular phenotype file
This workflow produce a bed+tabix file for all the molecular pheno data that are included in the region list to feed into downstream analysis

In [ ]:
[partition_by_chrom_1]
# An index text file with 5 columns specifying the chr, start, end and names of regions to analyze
parameter: region_list = path
# Path to the input molecular phenotype data.
input: phenoFile ,for_each = "chrom"
output: f'{cwd}/{name}.{_chrom}.mol_phe.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime,  mem = mem, tags = f'{step_name}_{_output:bn}'  
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container = container
    zcat $[_input] | head -1 > $[_output:n]
    tabix $[_input] $[_chrom] >> $[_output:n] 
    bgzip -f $[_output:n]
    tabix -p bed $[_output] -f

In [ ]:
[partition_by_chrom_2]
# Path to the input molecular phenotype data.
input: group_by = "all"
output: f'{cwd}/{name}.processed_phenotype.per_chrom.recipe'
import pandas as pd
chrom_df = pd.DataFrame({"#id" : chrom ,"#dir" : _input})
chrom_df.to_csv(_output,index = 0,sep = "\t")

In [ ]:
[partition_by_gene_1]
# An index text file with 5 columns specifying the chr, start, end and names of regions to analyze
parameter: region_list = path
regions = [x.strip().split() for x in open(region_list).readlines() if x.strip() and not x.strip().startswith('#')]
# Get the unique chormosome that have regions to be analyzed.
def extract(lst):
    return [item[0] for item in lst]
chrom = list(set(extract(regions)))
# Path to the input molecular phenotype data.
input: phenoFile ,for_each = "regions"
output: f'{cwd}/{name}.{_regions[3]}.{_regions[4]}.mol_phe.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime,  mem = mem, tags = f'{step_name}_{_output:bn}'  
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container = container
    zcat $[_input] | head -1 > $[_output:n]
    zcat $[_input] | grep  $[_regions[3] if gene_name_as_phenotype_id else _regions[4]] >> $[_output:n]
    bgzip -f $[_output:n]
    tabix -p bed $[_output] -f

In [ ]:
[partition_by_gene_2]
input: group_by = "all"
output: f'{cwd}/{name}.processed_phenotype.per_gene.recipe'
import pandas as pd
region_df = pd.DataFrame({"#id" : [x[3] for x in regions] ,"dir" : _input})
region_df.to_csv(_output,index = 0,sep = "\t")

In [ ]:
[bed_filter_na_mean_impute]
# Tolerance of missingness rows with missing rate larger than tol_missing will be removed,
# with missing rate smaller than tol_missing will be mean_imputed. Say if we want to keep rows with less than 5% missing, then we use 0.05 as tol_missing.
parameter: tol_missing = 0.05
input: phenoFile
output: f'{_input:nn}.filter_na.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
   library("dplyr")
   library("tibble")
   library("readr")
   compute_missing <- function(mtx){
          miss <- sum(is.na(mtx))/length(mtx)
          return(miss)
        }

        mean_impute <- function(mtx){
          f <- apply(mtx, 2, function(x) mean(x,na.rm = TRUE))
          for (i in 1:length(f)) mtx[,i][which(is.na(mtx[,i]))] <- f[i]
          return(mtx)
        }
    
        filter_mtx <- function(X, missing_rate_thresh) {
            rm_col <- which(apply(X, 2, compute_missing) > missing_rate_thresh)
            if (length(rm_col)) X <- X[, -rm_col]
            return(mean_impute(X))
        }  
  
    bed = read_delim("${_input}")
    mtx = bed[,5:ncol(bed)]%>%as.matrix
    rownames(mtx) = bed[,4]%>%unlist()
    tbl_filtered = filter_mtx(mtx%>%t(),${tol_missing})%>%t()%>%as_tibble(rownames = colnames(bed)[4] )
    bed_filtered = inner_join(bed[,1:4],tbl_filtered)
    bed_filtered%>%write_delim(${_output}, "\t" )
  
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    bgzip -f ${_output:n}
    tabix ${_output}

In [ ]:
[bed_filter_na]
# Tolerance of missingness rows with missing rate larger than tol_missing will be removed,
# with missing rate smaller than tol_missing will be mean_imputed. Say if we want to keep rows with less than 5% missing, then we use 0.05 as tol_missing.
parameter: tol_missing = 0.05
input: phenoFile
output: f'{_input:nn}.filter_na.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
   library("dplyr")
   library("tibble")
   library("readr")
   compute_missing <- function(mtx){
          miss <- sum(is.na(mtx))/length(mtx)
          return(miss)
        }

    
        filter_mtx <- function(X, missing_rate_thresh) {
            rm_col <- which(apply(X, 2, compute_missing) > missing_rate_thresh)
            if (length(rm_col)) X <- X[, -rm_col]
            return(X)
        }  
  
    bed = read_delim("${_input}", "\t")
    mtx = bed[,5:ncol(bed)]%>%as.matrix
    rownames(mtx) = bed[,4]%>%unlist()
    tbl_filtered = filter_mtx(mtx%>%t(),${tol_missing})%>%t()%>%as_tibble(rownames = colnames(bed)[4] )
    bed_filtered = inner_join(bed[,1:4],tbl_filtered)
    bed_filtered%>%write_delim("${_output:n}", "\t" )
  
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    bgzip -f ${_output:n}
    tabix ${_output} -f

In [ ]:
[per_chr_softImpute_1]
parameter: rank_max = 50 # max rank estimated in the per-chr methyl matrix
parameter: lambda_hyp = 30 # hyper par, indicating the importance of the nuclear norm
parameter: chr_recipe = path
chrom_path_ls = pd.read_csv(chr_recipe, sep = "\t")
per_chr_file = chrom_path_ls["#dir"].tolist()
input: for_each = "per_chr_file"
output: f'{_per_chr_file}_chr.softImputed.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library(softImpute)
    library(dplyr)
    library(readr)
    library(tibble)
    pheno = read_delim("${_per_chr_file}", delim = "\t")
    pheno_C=as(as.matrix(pheno[, 5:ncol(pheno)]),"Incomplete")
    fit=softImpute(pheno_C,rank=${rank_max},lambda=${lambda_hyp},type="svd")
    X_imp_all= complete(as.matrix(pheno),fit)
    as_tibble(X_imp_all) %>% write_delim("${_output:n}", "\t")
    
  
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    bgzip -f ${_output:n}
    tabix '${_output}' -f

In [ ]:
[per_chr_softImpute_2]
# Path to the input molecular phenotype data.
input: group_by = "all"
output: recipe = f'{cwd}/{name}.processed_phenotype.per_chrom.softImputed.recipe',
        imp_all = f'{cwd:a}/{phenoFile:bn}.softImputed.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container=container
    library(dplyr)
    library(readr)
    library(stringr)
    input = str_split("${_input}", " ")[[1]]
    recipe_tib = tibble('#dir' = input)
    recipe_tib = recipe_tib %>% mutate(`#chr` = str_extract(`#dir`, "(chr\\d+)|chrX")) %>% dplyr::select(c("#chr", "#dir"))
    write_delim(recipe_tib, delim = "\t", file = "${_output[0]}" )
    imp_all = NULL
    for(i in 1:length(input)){
      tmp = read_delim(input[i], "\t")
      imp_all = rbind(imp_all, tmp)
      rm(tmp)
      }
  
    imp_all %>% write_delim("${_output[1]:n}", "\t")
  
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    bgzip -f f'${_output[1]:n}'
    tabix ${_output[1]}

In [7]:
import os
string = path("home/a.bed.gz")
print(f'{string:n}')

home/a.bed


In [ ]:
[bam_subsetting]
parameter: region = "chr21 chr22"
input: phenoFile , group_by = 1
output: f'{cwd}/{_input:bn}.subsetted.bam'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    samtools view -b ${_input} ${region} > ${_output}

In [ ]:
[bam_to_fastq]
input: phenoFile, group_by = 1
output: f'{cwd}/{_input:bn}.1.fastq',f'{cwd}/{_input:bn}.2.fastq'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container=container
    samtools fastq -1 ${_output[0]} -2 ${_output[1]} ${_input}